## Joins

In [3]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("day3").getOrCreate()


23/09/04 14:03:05 WARN Utils: Your hostname, riyaz-Aspire-VX5-591G resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface enp3s0)
23/09/04 14:03:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/04 14:03:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable



Question: You are given two DataFrames: employees_df and departments_df, which contain information about employees and their respective departments. The schema for the DataFrames is as follows:

employees_df schema:
|-- employee_id: integer (nullable = true)
|-- employee_name: string (nullable = true)
|-- department_id: integer (nullable = true)

departments_df schema:

|-- department_id: integer (nullable = true)
|-- department_name: string (nullable = true)

Employees DataFrame:
                                                                                
+-----------+-------------+-------------+
|employee_id|employee_name|department_id|
+-----------+-------------+-------------+
|1          |Pallavi mam  |101          |
|2          |Bob          |102          |
|3          |Cathy        |101          |
|4          |David        |103          |
|5          |Amrit Sir    |104          |
|6          |Alice        |null         |
|7          |Eva          |null         |
|8          |Frank        |110          |
|9          |Grace        |109          |
|10         |Henry        |null         |
+-----------+-------------+-------------+



Departments DataFrame:
+-------------+------------------------+
|department_id|department_name         |
+-------------+------------------------+
|101          |HR                      |
|102          |Engineering             |
|103          |Finance                 |
|104          |Marketing               |
|105          |Operations              |
|106          |null                    |
|107          |Operations              |
|108          |Production              |
|null         |Finance                 |
|110          |Research and Development|
+-------------+----------------------


In [4]:
from pyspark.sql import Row

# Create a SparkSession
# spark = SparkSession.builder \
#     .appName("CreateDataFrames") \
#     .getOrCreate()

# Create the Employees DataFrame
employees_data = [
    Row(employee_id=1, employee_name="Pallavi mam", department_id=101),
    Row(employee_id=2, employee_name="Bob", department_id=102),
    Row(employee_id=3, employee_name="Cathy", department_id=101),
    Row(employee_id=4, employee_name="David", department_id=103),
    Row(employee_id=5, employee_name="Amrit Sir", department_id=104),
    Row(employee_id=6, employee_name="Alice", department_id=None),
    Row(employee_id=7, employee_name="Eva", department_id=None),
    Row(employee_id=8, employee_name="Frank", department_id=110),
    Row(employee_id=9, employee_name="Grace", department_id=109),
    Row(employee_id=10, employee_name="Henry", department_id=None)
]

employees_df = spark.createDataFrame(employees_data)

# Show the Employees DataFrame
print("Employees DataFrame:")
employees_df.show()

Employees DataFrame:


+-----------+-------------+-------------+
|employee_id|employee_name|department_id|
+-----------+-------------+-------------+
|          1|  Pallavi mam|          101|
|          2|          Bob|          102|
|          3|        Cathy|          101|
|          4|        David|          103|
|          5|    Amrit Sir|          104|
|          6|        Alice|         null|
|          7|          Eva|         null|
|          8|        Frank|          110|
|          9|        Grace|          109|
|         10|        Henry|         null|
+-----------+-------------+-------------+



In [5]:
# Create the Departments DataFrame
departments_data = [
    Row(department_id=101, department_name="HR"),
    Row(department_id=102, department_name="Finance"),
    Row(department_id=103, department_name="Marketing"),
    Row(department_id=104, department_name="IT"),
    Row(department_id=105, department_name="Sales"),
    Row(department_id=106, department_name="Engineering"),
    Row(department_id=107, department_name="Legal"),
    Row(department_id=108, department_name="Operations"),
    Row(department_id=109, department_name="Customer Support"),
    Row(department_id=110, department_name="Management")
]

departments_df = spark.createDataFrame(departments_data)

# Show the Departments DataFrame
print("Departments DataFrame:")
departments_df.show()




Departments DataFrame:
+-------------+----------------+
|department_id| department_name|
+-------------+----------------+
|          101|              HR|
|          102|         Finance|
|          103|       Marketing|
|          104|              IT|
|          105|           Sales|
|          106|     Engineering|
|          107|           Legal|
|          108|      Operations|
|          109|Customer Support|
|          110|      Management|
+-------------+----------------+



In [6]:
# Creating temporary view for both DF

departments_df.createOrReplaceTempView("department")
employees_df.createOrReplaceTempView("employee")

### Join Expressions

Question: How can you combine the employees_df and departments_df DataFrames based on the common "department_id" column to get a combined DataFrame with employee names and their respective department names?

In [5]:
# pyspark

joinExpression = departments_df['department_id'] == employees_df['department_id']

departments_df.join(employees_df, joinExpression).select(departments_df['department_id'],employees_df['employee_id'],employees_df['employee_name'],departments_df['department_name']).show()

+-------------+-----------+-------------+----------------+
|department_id|employee_id|employee_name| department_name|
+-------------+-----------+-------------+----------------+
|          101|          1|  Pallavi mam|              HR|
|          101|          3|        Cathy|              HR|
|          102|          2|          Bob|         Finance|
|          103|          4|        David|       Marketing|
|          104|          5|    Amrit Sir|              IT|
|          109|          9|        Grace|Customer Support|
|          110|          8|        Frank|      Management|
+-------------+-----------+-------------+----------------+



In [7]:
# sparksql

join_exp = spark.sql("""
                    select d.department_id, e.employee_id, e.employee_name, d.department_name
                    from department d
                    join employee e
                    on d.department_id = e.department_id
""")
                     
join_exp.show()

+-------------+-----------+-------------+----------------+
|department_id|employee_id|employee_name| department_name|
+-------------+-----------+-------------+----------------+
|          101|          1|  Pallavi mam|              HR|
|          101|          3|        Cathy|              HR|
|          102|          2|          Bob|         Finance|
|          103|          4|        David|       Marketing|
|          104|          5|    Amrit Sir|              IT|
|          109|          9|        Grace|Customer Support|
|          110|          8|        Frank|      Management|
+-------------+-----------+-------------+----------------+



### Inner Joins

Question: How can you retrieve employee names and their respective department names for employees belonging to the "Engineering" department?

In [16]:
# pyspark

joinExpression = departments_df['department_id'] == employees_df['department_id']
engg_exp = departments_df['department_name'] == 'Engineering'

departments_df.join(employees_df, joinExpression).select(employees_df['employee_name'],departments_df['department_name']).where(engg_exp).show()

+-------------+---------------+
|employee_name|department_name|
+-------------+---------------+
+-------------+---------------+



In [10]:
# sparksql

inner_join_sql = spark.sql("""
                            select e.employee_name, d.department_name
                           from department d
                           join employee e
                           on e.department_id = d.department_id
                           where d.department_name = 'Engineering' 
 """)

inner_join_sql.show()

+-------------+---------------+
|employee_name|department_name|
+-------------+---------------+
+-------------+---------------+



### Outer Joins

Question: Retrieve a DataFrame that contains all employees along with their department names. If an employee doesn't have a department assigned, display "No Department".

In [17]:
# pyspark

joinType = 'outer'

#Outer joined two dataframes which returned every value possible and null if any value is not present
outer_joined_df = departments_df.join(employees_df, joinExpression, joinType).select(employees_df['employee_name'],departments_df['department_name'])

#filled null with default value as No Department
No_dept_filled = outer_joined_df.na.fill('No Department')
No_dept_filled.show()


+-------------+----------------+
|employee_name| department_name|
+-------------+----------------+
|        Alice|   No Department|
|          Eva|   No Department|
|        Henry|   No Department|
|        Cathy|              HR|
|  Pallavi mam|              HR|
|          Bob|         Finance|
|        David|       Marketing|
|    Amrit Sir|              IT|
|No Department|           Sales|
|No Department|     Engineering|
|No Department|           Legal|
|No Department|      Operations|
|        Grace|Customer Support|
|        Frank|      Management|
+-------------+----------------+



In [11]:
# sparksql

outer_join_sql = spark.sql("""
                            select coalesce(e.employee_name, 'no Employee') as employee_name, 
                                   coalesce(d.department_name, 'no department') as department_name
                           from employee e
                           full outer join department d
                           on d.department_id = e.department_id
"""
)

outer_join_sql.show()

+-------------+----------------+
|employee_name| department_name|
+-------------+----------------+
|        Alice|   no department|
|          Eva|   no department|
|        Henry|   no department|
|        Cathy|              HR|
|  Pallavi mam|              HR|
|          Bob|         Finance|
|        David|       Marketing|
|    Amrit Sir|              IT|
|  no Employee|           Sales|
|  no Employee|     Engineering|
|  no Employee|           Legal|
|  no Employee|      Operations|
|        Grace|Customer Support|
|        Frank|      Management|
+-------------+----------------+



### Left Outer Joins

Question: List all employees along with their department names. If an employee doesn't have a department assigned, display "No Department".

In [18]:
# pyspark

joinType = 'left_outer'
employees_df.join(departments_df,joinExpression,joinType).select(employees_df['employee_name'],departments_df['department_name']).na.fill('no Department').show()

+-------------+----------------+
|employee_name| department_name|
+-------------+----------------+
|  Pallavi mam|              HR|
|          Bob|         Finance|
|        David|       Marketing|
|        Cathy|              HR|
|        Alice|   no Department|
|    Amrit Sir|              IT|
|        Frank|      Management|
|          Eva|   no Department|
|        Henry|   no Department|
|        Grace|Customer Support|
+-------------+----------------+



In [12]:
# sparksql

left_outer_join_sql = spark.sql("""
                                    select e.employee_name,
                                            coalesce(d.department_name, 'no department') as department_name
                                    from employee e
                                    left outer join department d
                                    on e.department_id = d.department_id       

                                """)

left_outer_join_sql.show()

+-------------+----------------+
|employee_name| department_name|
+-------------+----------------+
|  Pallavi mam|              HR|
|          Bob|         Finance|
|        David|       Marketing|
|        Cathy|              HR|
|        Alice|   no department|
|    Amrit Sir|              IT|
|        Frank|      Management|
|          Eva|   no department|
|        Henry|   no department|
|        Grace|Customer Support|
+-------------+----------------+



### Right Outer Joins

Question: Display a list of departments along with employee names. If a department has no employees, display "No Employees".



In [19]:
# pyspark

joinType = 'right_outer'

employees_df.join(departments_df,joinExpression,joinType).select(departments_df['department_name'],employees_df['employee_name']).na.fill('No Employee', subset=['employee_name']).show()

+----------------+-------------+
| department_name|employee_name|
+----------------+-------------+
|              HR|        Cathy|
|              HR|  Pallavi mam|
|         Finance|          Bob|
|       Marketing|        David|
|              IT|    Amrit Sir|
|     Engineering|  No Employee|
|           Sales|  No Employee|
|      Management|        Frank|
|           Legal|  No Employee|
|      Operations|  No Employee|
|Customer Support|        Grace|
+----------------+-------------+



In [13]:
# sparksql

right_outer_join_sql = spark.sql("""
                                select d.department_name as department_name,
                                       coalesce(e.employee_name,'No employee') as employee_name
                                 from employee e
                                 right outer join department d
                                 on e.department_id = d.department_id
                                """)

right_outer_join_sql.show()

+----------------+-------------+
| department_name|employee_name|
+----------------+-------------+
|              HR|        Cathy|
|              HR|  Pallavi mam|
|         Finance|          Bob|
|       Marketing|        David|
|              IT|    Amrit Sir|
|     Engineering|  No employee|
|           Sales|  No employee|
|      Management|        Frank|
|           Legal|  No employee|
|      Operations|  No employee|
|Customer Support|        Grace|
+----------------+-------------+



### Left Semi Joins

Question: Retrieve a DataFrame that includes employee names for departments that have employees.



In [21]:
# pyspark

joinType = 'left_semi'

employees_df.join(departments_df,joinExpression,joinType).select(employees_df['employee_name']).show()

+-------------+
|employee_name|
+-------------+
|  Pallavi mam|
|        Cathy|
|          Bob|
|        David|
|    Amrit Sir|
|        Grace|
|        Frank|
+-------------+



In [14]:
# sparksql

left_semi_join_sql = spark.sql("""
                                select e.employee_name
                               from employee e
                               left semi join department d
                               on e.department_id = d.department_id
                                
                               """ )

left_semi_join_sql.show()

+-------------+
|employee_name|
+-------------+
|  Pallavi mam|
|        Cathy|
|          Bob|
|        David|
|    Amrit Sir|
|        Grace|
|        Frank|
+-------------+



### Left Anti Joins

Question: Find the employees who don't belong to any department.

In [22]:
# pyspark

joinType = 'left_anti'

employees_df.join(departments_df,joinExpression,joinType).select(employees_df['employee_name']).show()

+-------------+
|employee_name|
+-------------+
|        Alice|
|          Eva|
|        Henry|
+-------------+



In [15]:
# sparksql

left_anti_join_sql = spark.sql("""
                                select e.employee_name
                               from employee e
                               left anti join department d
                               on e.department_id = d.department_id
                                
                               """ )

left_anti_join_sql.show()

+-------------+
|employee_name|
+-------------+
|        Alice|
|          Eva|
|        Henry|
+-------------+



### Cross (Cartesian) Joins

Question: Create a DataFrame that contains all possible combinations of employees and departments.

In [23]:
# pyspark

joinType = 'cross'
employees_df.crossJoin(departments_df).show()

+-----------+-------------+-------------+-------------+----------------+
|employee_id|employee_name|department_id|department_id| department_name|
+-----------+-------------+-------------+-------------+----------------+
|          1|  Pallavi mam|          101|          101|              HR|
|          1|  Pallavi mam|          101|          102|         Finance|
|          2|          Bob|          102|          101|              HR|
|          2|          Bob|          102|          102|         Finance|
|          1|  Pallavi mam|          101|          103|       Marketing|
|          1|  Pallavi mam|          101|          104|              IT|
|          2|          Bob|          102|          103|       Marketing|
|          2|          Bob|          102|          104|              IT|
|          1|  Pallavi mam|          101|          105|           Sales|
|          1|  Pallavi mam|          101|          106|     Engineering|
|          2|          Bob|          102|          

In [16]:
# sparksql

cross_join_sql = spark.sql("""
                            select * 
                           from employee
                           cross join department
""")
                           
cross_join_sql.show()

+-----------+-------------+-------------+-------------+----------------+
|employee_id|employee_name|department_id|department_id| department_name|
+-----------+-------------+-------------+-------------+----------------+
|          1|  Pallavi mam|          101|          101|              HR|
|          1|  Pallavi mam|          101|          102|         Finance|
|          2|          Bob|          102|          101|              HR|
|          2|          Bob|          102|          102|         Finance|
|          1|  Pallavi mam|          101|          103|       Marketing|
|          1|  Pallavi mam|          101|          104|              IT|
|          2|          Bob|          102|          103|       Marketing|
|          2|          Bob|          102|          104|              IT|
|          1|  Pallavi mam|          101|          105|           Sales|
|          1|  Pallavi mam|          101|          106|     Engineering|
|          2|          Bob|          102|          